In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
import re
from cleantext import clean
import string
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
#read in dataframe 
df=pd.read_excel('Copy of Bot Data.xlsx')
df.head()

,Username,Follower Count,Following Count,Follower/Following Ratio,Number of posts,biography,# of likes on comment,comment,time of comment after post (minutes),"bot classification (0-not a bot, 1-bot)"
0,alyseaa_kyyy,28,405,0.069136,5,MORE VIDEOS UNCENSORED\n👇👇👇\ncompeten-hodgkin....,0,Don't look at my story!!!🔥🔥🔥,8,1
1,bellafreakyxo,858,834,1.028777,3,"bella🧸 she/xe\n19 | Florida 🌴\nMessage me ""Sen...",0,"Dm me for a fr***y vid, and tell me how long y...",1,1
2,jeniver.f,16,131,0.122137,0,jeniver.f\n⤵️⤵️⤵️⤵️\nsoft-rolypoly-fcc1ec.netl...,0,Don't judge me after u saw my stories😭😭😭,1,1
3,deanna__ward,244,289,0.844291,8,Deanna Ward\nHOT STORY🔞\n✨💕 Stockings & Heels💕...,0,😍I am always looking forward to your posts😍,1,1
4,briana.nastyy,32,155,0.206452,1,briana.nastyy\npolite-lily-69612e.netlify.app,1,Don't judge me after view my stories babe..😭😭😭,1,1


In [3]:
#text preprocessing

#convert text to lower case 
df['biography']=df['biography'].str.lower()
df['comment']=df['comment'].str.lower()

#expand contractions 
contractions_dict = {"aren't": "are not", "don't": "do not", "Don't": "do not", "I'm": "I am", "i'm": "I am", 
                    "it's": "it is", "y'all": "you all", "Y'all": "you all", "didn't": "did not", "won't": "will not",
                   "I'll": "I will", "i'll": "I will", "can't": "can not"}
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text, expand_dict):
    def replace(match):
        return expand_dict[match.group(0)]
    return contractions_re.sub(replace, text)
df['biography']=df['biography'].apply(lambda x:expand_contractions(x, contractions_dict ))
df['comment']=df['comment'].apply(lambda x:expand_contractions(x, contractions_dict))

In [4]:
#convert text to features 
bios=df['biography'].astype(str)
usernames=df['Username']
comments=df['comment']

In [5]:
#filter out emojis 
new_bios=[]
for b in bios: 
    temp=clean(b, no_emoji=True)
    temp=temp.replace("\n", " ")
    if temp=='':
        new_bios.append('None')
    else: 
        new_bios.append(temp)
new_comments=[]
for c in comments:
    temp=clean(c, no_emoji=True)
    temp=temp.replace("\n", " ")
    if temp=='':
        new_comments.append('None')
    else: 
        new_comments.append(temp)
df_filtered=df.copy()
df_filtered['biography']=new_bios
df_filtered['comment']=new_comments

In [6]:
#train and test set split 
train_set, test_set= train_test_split(
df_filtered, test_size=.2, random_state=0)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

#classifying the biographies 
vectorizer1 = CountVectorizer(min_df=0, lowercase=False)
vectorizer1.fit(train_set['biography'])
bio_array = vectorizer1.transform(train_set['biography']).toarray()
bio_df = pd.DataFrame(data=bio_array,columns = vectorizer1.get_feature_names())

In [8]:
#classifying the comments 
vectorizer2 = CountVectorizer(min_df=0, lowercase=False)
vectorizer2.fit(train_set['comment'])
comment_array=vectorizer2.transform(train_set['comment']).toarray()
comment_df = pd.DataFrame(data=comment_array,columns = vectorizer2.get_feature_names())

In [21]:
#normalize training data set 
train_set_filtered=train_set[['Username', 'Follower Count', 'Following Count',
       'Follower/Following Ratio', 'Number of posts', '# of likes on comment ', 'time of comment after post (minutes)', 'bot classification (0-not a bot, 1-bot)']]
df_normalized=train_set_filtered.copy()
numerical_cols=['Follower Count', 'Following Count',
       'Follower/Following Ratio', 'Number of posts', '# of likes on comment ', 'time of comment after post (minutes)']
for c in numerical_cols:
    df_normalized[c]= (train_set_filtered[c] - train_set_filtered[c].min()) / (train_set_filtered[c].max() - train_set_filtered[c].min())
    
#merge normalized numerical values with dataframes containing text classification features 
df_normalized=df_normalized.reset_index(drop=True)
df_combined=pd.concat([comment_df, bio_df],axis=1)
df_combined 
df_combined=pd.concat([df_combined, df_normalized], axis=1)
df_combined.replace([np.inf, -np.inf], np.nan, inplace=True)
df_combined=df_combined.set_index('Username')
df_combined=df_combined.dropna()
df_combined
X_train=df_combined.drop(['bot classification (0-not a bot, 1-bot)'], axis=1)
Y_train=df_combined['bot classification (0-not a bot, 1-bot)']


,000,10,1000,11,12,13,15,1500,16,19,...,youtu,youtube,yt,yuhs,zainab,zainabzilahi,zeldon,zilahi,zooming,zu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#normalize the test set 
test_normalized=test_set.copy()
test_normalized=test_normalized[['Username','Follower Count', 'Following Count',
       'Follower/Following Ratio', 'Number of posts', '# of likes on comment ', 'time of comment after post (minutes)', 'bot classification (0-not a bot, 1-bot)']]
for c in numerical_cols: 
    test_normalized[c]= (test_normalized[c] - test_normalized[c].min()) / (test_normalized[c].max() - test_normalized[c].min())  
test_normalized=test_normalized.reset_index(drop=True)

In [11]:
#convert the text columns to arrays and merge dataframes 
bio_array_test = vectorizer1.transform(test_set['biography']).toarray()
bio_df_test = pd.DataFrame(data=bio_array_test,columns = vectorizer1.get_feature_names())
comment_array_test=vectorizer2.transform(test_set['comment']).toarray()
comment_df_test = pd.DataFrame(data=comment_array_test,columns = vectorizer2.get_feature_names())
df_combined_test=pd.concat([comment_df_test, bio_df_test],axis=1)
df_combined_test=pd.concat([df_combined_test, test_normalized], axis=1)
df_combined_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_combined_test=df_combined_test.dropna()
df_combined_test=df_combined_test.set_index('Username')
X_test=df_combined_test.drop(['bot classification (0-not a bot, 1-bot)'], axis=1)
Y_test=df_combined_test['bot classification (0-not a bot, 1-bot)']
X_test

,000,10,1000,11,12,13,15,1500,16,19,...,zeldon,zilahi,zooming,zu,Follower Count,Following Count,Follower/Following Ratio,Number of posts,# of likes on comment,time of comment after post (minutes)
Username,,,,,,,,,,,,,,,,,,,,,
earlenemega,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.000074,0.081957,0.000138,0.000000,0.030170,0.000000
kidjackpot1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.000091,0.022018,0.000632,0.014327,0.001257,0.075472
richard_westlife_sfxo,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.003921,0.848930,0.000709,0.000805,0.000000,0.018868
imalex204__,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.000348,0.028135,0.001901,0.000161,0.001886,0.094340
juliabielza,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.001110,0.017125,0.009955,0.000966,0.044626,0.113208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doogleass92,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.002691,0.373089,0.001107,0.072601,0.231615,0.000000
_nickkkk0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.001606,0.151070,0.001632,0.000322,0.000629,0.188679
marie_uya2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.000088,0.066055,0.000204,0.000000,0.000000,0.018868


In [12]:
#Logistic Regression 
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, Y_train)
score = classifier.score(X_test, Y_test)
print("Accuracy:", score)
LR_prediction=classifier.predict(X_test)
print(confusion_matrix(LR_prediction, Y_test))
print(classification_report(LR_prediction, Y_test))

Accuracy: 0.9206349206349206
[[25  5]
 [ 0 33]]
              precision    recall  f1-score   support

           0       1.00      0.83      0.91        30
           1       0.87      1.00      0.93        33

    accuracy                           0.92        63
   macro avg       0.93      0.92      0.92        63
weighted avg       0.93      0.92      0.92        63



In [15]:
#trying other classification algorithms 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

#SVC, KNN, DTC, Naive Bayes
SVC_model = SVC()
KNN_model = KNeighborsClassifier(n_neighbors=5)
Bayes_model = GaussianNB()
dtc = tree.DecisionTreeClassifier()

SVC_model.fit(X_train, Y_train)
KNN_model.fit(X_train, Y_train)
Bayes_model.fit(X_train, Y_train)
dtc.fit(X_train, Y_train)

SVC_prediction = SVC_model.predict(X_test)
KNN_prediction = KNN_model.predict(X_test)
Bayes_prediction=Bayes_model.predict(X_test)  
DTC_prediction = dtc.predict(X_test)

print(accuracy_score(SVC_prediction, Y_test))
print(accuracy_score(KNN_prediction, Y_test))
print(accuracy_score(Bayes_prediction, Y_test))
print(accuracy_score(DTC_prediction, Y_test))
#logistic regression is still the most accurate! 

0.9206349206349206
0.8253968253968254
0.8888888888888888
0.8888888888888888


In [18]:
#looking at feature importance- five most important features
importance = final_model.coef_[0]
indices = (-importance).argsort()[:5]
col_names=X_test.columns.tolist()
important_features = [col_names[i] for i in indices]
important_features

['shy', 'laugh', 'velisabelli775', 'tapmybio', 'bit']